In [24]:
from osgeo import gdal
import numpy
from numpy.core.umath import add, subtract


class MDimage(object):
    def __init__(self, filepath):
        self.filepath = filepath
        self.ds = gdal.Open(filepath, gdal.GA_ReadOnly)
        self.iminfo = dict()
        self.iminfo['bandnum'] = self.ds.RasterCount
        self.iminfo['cols'] = self.ds.RasterXSize
        self.iminfo['rows'] = self.ds.RasterYSize
        self.iminfo['originX'] = self.ds.GetGeoTransform()[0]
        self.iminfo['originY'] = self.ds.GetGeoTransform()[3]
        self.iminfo['pixelWidth'] = self.ds.GetGeoTransform()[1]
        self.iminfo['pixelHeight'] = self.ds.GetGeoTransform()[5]

class Index(MDimage):
    def __init__(self, filepath):
        self.filepath = filepath
        super(Index, self).__init__(filepath)

    def Band2Array(self,factor,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        self.fill=fill
        array = band.ReadAsArray(0, 0, cols, rows)
        #array [array <-100] =fill
        #array = numpy.where(array ==fill, fill , array * 0.0001)
        array= array.reshape(cols*rows)
        return  array

    def trtData(self,evi,lswi,dvel):
        self.evi=evi
        self.lswi=lswi
        self.dvel=dvel
        X=[]
        for i in range(len(evi)):
            F=[]
            F.append(evi[i])
            F.append(lswi[i])
            F.append(dvel[i])
            #F.append(subtract(evi[i],lswi[i]))
            X.append(F)
        return X

    def trainResult(self,flood):
        self.flood=flood
        Y=[]
        for i in range(len(flood)):
            Y.append(flood[i])
        return Y

    def index2Array(self,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        self.fill=fill
        array = band.ReadAsArray(0, 0, cols, rows)
        array = array.reshape(cols * rows)
        return  array

    def csv2Array(self,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        path = "/home/faizan/Desktop/full/result.csv"
        data = numpy.genfromtxt(path, dtype=float, delimiter=',')
        array = data.reshape(cols * rows)
        return  array


    def WriteArrayAsImage(self, out_fname, outArray):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        driver = self.ds.GetDriver()
        self.outArray = outArray
        outArray= outArray.reshape([rows,cols])
        outDS = driver.Create(out_fname, cols, rows, 1, gdal.GDT_Float32)
        outDS.SetGeoTransform(self.ds.GetGeoTransform())
        outDS.SetProjection(self.ds.GetProjection())
        outBand = outDS.GetRasterBand(1)
        outBand.WriteArray(outArray)
        outDS = None
        del outDS, outBand
    
    
    def cMatrix(self, actual, test):
        self.actual=actual
        self.test=test
        F_F=0
        F_N=0
        F_fi=0
        N_F =0
        N_N =0
        N_fi=0
        Fi_F=0
        Fi_N=0
        Fi_fi=0
    
        for i in range(len(actual)):
            if actual[i]==test[i]:
                if actual[i]==0:
                    N_N += 1
                elif actual[i]==1:
                    F_F += 1
                else:
                    Fi_fi += 1
            else:
                if actual[i]==0:
                    if test[i]==1:
                        N_F += 1
                    else:
                        N_fi+= 1
                elif actual[i]==1:
                    if test[i]==0:
                        F_N +=1
                    else:
                        F_fi +=1 
                else:
                    if test[i]==1:
                        Fi_F +=1
                    else:
                        Fi_N +=1
        
        LIST= [F_F,F_N,F_fi,N_F,N_N,
               N_fi,Fi_F,Fi_N,Fi_fi]
        
        C_matrix = numpy.array(LIST).reshape(3,3)
        return C_matrix
    
    


In [39]:

import numpy as np
from sklearn import svm
from osgeo import gdal
from sklearn.externals import joblib
#from multiprocessing import Pool

fill=-999
path="/home/faizan/USA_data/h11v05/"

#assigning names test data
evi=path+"A2008161.US.evi.tif"
dvel=path+"A2008161.US.dvel.tif"
lswi=path+"A2008161.US.lswi.tif"
Oflood = path+"A2008161.US.machineflood.tif"
#save model

#joblib.dump(clf, '/home/faizan/Desktop/full/trained.pkl')
clf = joblib.load('/home/faizan/Desktop/full/trained.pkl') 

#del EVI_t, LSWI_t, DVEL_t, Flood_t, X, Y
#Test data
limage = Index(evi)
cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
EVI = (limage.index2Array(fill))

limage = Index(lswi)
LSWI = (limage.index2Array(fill))

limage = Index(dvel)
DVEL = (limage.index2Array(fill))

data = (limage.trtData(EVI, LSWI, DVEL))

#pool = Pool(processess = 5)
#flood_array = pool.map(work, (data/5, data/5, data/5, data/5, data/5)
#Get Flood as tiff
#flood = clf.predict(data)
limage.WriteArrayAsImage(Oflood, flood)
#del flood, EVI, LSWI, DVEL, data


In [45]:
len(data)/5



1768800

In [46]:
data[1777000]

[-999.0, -999.0, -999.0]

In [34]:
fill=-999
Actual = "/home/faizan/Desktop/full/2010249act_C.tif"
limage = Index(Actual)
cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
Actual = (limage.index2Array(fill))


In [29]:
Actual

array([-999., -999., -999., ..., -999., -999., -999.], dtype=float32)

In [35]:
test = "/home/faizan/Desktop/full/2010249res_C.tif"
limage = Index(test)
cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']
test = (limage.index2Array(fill))

In [36]:
test

array([-999., -999., -999., ..., -999., -999., -999.], dtype=float32)

In [37]:
cMat= limage.cMatrix(Actual, test)
cMat

array([[  70749,    2309,       0],
       [   2248,  502907, 1195120],
       [      0,       0,   66535]])

In [32]:
cMat= limage.cMatrix(Actual, test)
cMat

array([[1102000,   36105,       0],
       [  11732, 2799240,       0],
       [    407,       0, 2842516]])

In [21]:
cMat= limage.cMatrix(Actual, test)
cMat

array([[1067822,   47039,       0],
       [   9312, 3267721,       0],
       [      5,       0, 2400101]])

In [46]:
Mat= limage.cMatrix(Actual, flood)

In [47]:
Mat

array([[1893392,   52525,       0],
       [   5174, 2945136,       0],
       [      1,       0, 3947772]])

In [32]:
def trtData(self,actual, test):
    self.flood=flood
    self.test=test
    F_F=0
    F_N=0
    F_fi=0
    N_F =0
    N_N =0
    N_fi=0
    Fi_F=0
    Fi_N=0
    Fi_fi=0
    
    for i in range(len(actual)):
        if actual[i]==test[i]:
            if actual[i]==0:
                N_N += 1
            elif actual[i]==1:
                F_F += 1
            else:
                Fi_fi += 1
        else:
            if actual[i]==0:
                if test[i]==1:
                    N_F += 1
                else:
                    N_fi+= 1
            elif actual[i]==1:
                if test[i]==0:
                    F_N +=1
                else:
                    F_fi +=1 
            else:
                if test[i]==1:
                    Fi_F +=1
                else:
                    Fi_N +=1
    LIST= [F_F,
           F_N,
           F_fi,
           N_F,
           N_N,
           N_fi,
           Fi_F,
           Fi_N,
           Fi_fi]
    C_matrix = np.array(LIST).reshape(3,3)
    return C_matrix
  

In [30]:

import numpy as np

LIST= [1,
       2,
       3,
       4,
       5,
       5,
       5,
       5,
       6]
L = np.array(LIST).reshape(3,3)
L[1][0]



4

In [22]:
import pandas as pd
from osgeo import gdal

ds = gdal.Open("/home/faizan/Desktop/full/A2014257.evi.tif", gdal.GA_ReadOnly)
band = ds.GetRasterBand(1)
cols =ds.RasterXSize
rows =ds.RasterYSize
array = band.ReadAsArray(0, 0, cols, rows)
array = array.reshape(cols * rows)
len(array)

6792000

In [1]:
import random
import time
import sys
from multiprocessing import Pool
random.seed()
def genList (size):
    randomList = []
     
    #initialize random list with values between 0 and 100
    for i in range(size):
        randomList.append(random.randint(0,10))
         
    return randomList
#return the sum of all elements in the list
#This is the same as "return sum(inList)" but in long form for readability and emphasis
def sumList(inList):
    finalSum = 0
     
    #iterate over all values in the list, and calculate the cummulative sum
    for value in inList:
        finalSum = finalSum + value
    return finalSum
def doWork(N):
    #create a random list of N integers
    myList = genList (N)
    finalSum = sumList(myList)
    return finalSum
if __name__ == '__main__':
    if len(sys.argv) == 2 and sys.argv[1].isdigit():
        N = int(sys.argv[1])
        #mark the start time
        startTime = time.time()
         
        #create a process Pool with 4 processes
        pool = Pool(processes=4)
         
        #map doWork to availble Pool processes
        results = pool.map(doWork, (N/4,N/4,N/4,N/4))
         
        #sum the partial results to get the final result
        finalSum = sumList(results)
                 
        #mark the end time
        endTime = time.time()
        #calculate the total time it took to complete the work
        workTime =  endTime - startTime
         
        #print results
        print "The job took " + str(workTime) + " seconds to complete"
        print "The final sum was: " + str(finalSum)
    else:
        exit(-1)

In [1]:
n=2 
print n*2

4


In [1]:
import random
import time
import sys
from multiprocessing import Process, Queue
random.seed()
def genList (size):
    randomList = []
     
    #initialize random list with values between 0 and 100
    for i in range(size):
        randomList.append(random.randint(0,10))
         
    return randomList
#return the sum of all elements in the list
#This is the same as "return sum(inList)" but in long form for readability and emphasis
def sumList(inList):
    finalSum = 0
     
    #iterate over all values in the list, and calculate the cummulative sum
    for value in inList:
        finalSum = finalSum + value
    return finalSum
def doWork(N,q):
    #create a random list of N integers
    myList = genList (N)
    finalSum = sumList(myList)
    #put the result in the Queue to return the the calling process
    q.put(finalSum)
if __name__ == '__main__':
    if len(sys.argv) == 2 and sys.argv[1].isdigit():
        N = int(sys.argv[1])
        #mark the start time
        startTime = time.time()
        #create a Queue to share results
        q = Queue()
        #create 4 sub-processes to do the work
        p1 = Process(target=doWork, args=(N/4,q))
        p1.start()
        p2 = Process(target=doWork, args=(N/4,q))
        p2.start()
        p3 = Process(target=doWork, args=(N/4,q))
        p3.start()
        p4 = Process(target=doWork, args=(N/4,q))
        p4.start()
         
        results = []
        #grab 4 values from the queue, one for each process
        for i in range(4):
            #set block=True to block until we get a result
            results.append(q.get(True))
         
        #sum the partial results to get the final result
        finalSum = sumList(results)
         
        p1.join()
        p2.join()
        p3.join()
        p4.join()
                 
        #mark the end time
        endTime = time.time()
        #calculate the total time it took to complete the work
        workTime =  endTime - startTime
         
        #print results
        print "The job took " + str(workTime) + " seconds to complete"
        print "The final sum was: " + str(finalSum)
    else:
        exit(-1)